<font size=100>库+嵌入模型+llm</font>


In [ ]:
#配置环境，前面已经配置，这部分不需要重复运行
# %%capture --no-stderr
# %pip install -U langchain langchain_community pypdf sentence_transformers faiss
# %pip install jieba
# %pip install rank_bm25

In [2]:
# 导入必要的库并打印版本信息
import langchain, langchain_community, pypdf, sentence_transformers

for module in (langchain, langchain_community, pypdf, sentence_transformers):
    print(f"{module.__name__:<30}{module.__version__}")

#导入操作系统和数据处理相关库
import os
import pandas as pd


# 定义嵌入模型
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
import torch
from langchain_ollama import OllamaLLM
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')
#定义嵌入模型，跑通这部分代码需要开代理
embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-large-zh-v1.5',
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True}
)

#加载向量数据库
# 加载 index.faiss 文件作为向量数据库
from langchain_community.vectorstores.faiss import FAISS

import faiss
import os
import numpy as np
# 确保路径正确数据库文件\index.faiss
index_path = r"E:\RAG\database"
if os.path.exists(index_path):
    print("index.faiss 文件存在")
    vector_db = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)
    
# 输出向量数据库vector_db中包含的数据条数
print(f"向量数据库中数据总数: {vector_db.index.ntotal}")


#采用ollama的LLM模型千问
llm = OllamaLLM(model="qwen:7b")

langchain                     0.3.25
langchain_community           0.3.23
pypdf                         5.4.0
sentence_transformers         4.1.0
device: cpu
index.faiss 文件存在
向量数据库中数据总数: 759


<font color=pink size=100>简单检索<font>

In [10]:
# 在简单的数据检索基础上进行打分操作，采用similarity_search_with_score 方法
# 目前最低分数：0.8534546494483948
# similarity_search_with_score 方法不仅允许您返回文档，还允许返回查询到查询向量和文档向量之间的距离分数。返回的距离分数是L2距离。因此，分数越低越好。
# L2距离分数指的是欧氏距离（Euclidean Distance），也叫L2范数。在向量检索中，L2距离用于衡量两个向量（如查询向量和文档向量）之间的相似度。
def rag_search_with_score(query, k):
    """
    使用 vector_db 对输入 query 进行检索，返回前 k 条结果及其相似度分数。
    参数:
        query (str): 用户输入的检索问题。
        k (int): 返回的结果数量，默认为 5。
    返回:
        list: 每个元素为 dict，包含文档内容、元数据和相似度分数。
    """
    # 调用 vector_db 的 similarity_search_with_score 方法，
    # 返回 [(Document, score), ...]，每个元素是文档和对应的相似度分数
    results = vector_db.similarity_search_with_score(query, k=k)
    scored_docs = []  # 用于存储带分数的检索结果
    for doc, score in results:
        # 将分数写入文档的元数据，方便后续使用
        doc.metadata["l2_score"] = score
        # 构造包含内容、元数据和分数的字典，加入结果列表
        scored_docs.append({
            "content": doc.page_content,  # 文档内容
            "metadata": doc.metadata,     # 文档元数据（包含 l2_score）
            "l2_score": score            # L2 距离分数
        })
    return scored_docs  # 返回带分数的检索结果列表

# 示例调用
query = "请问保险产品有哪些？"  # 定义检索问题
k = 50  # 设置返回结果数量
# 调用函数 rag_search_with_score 进行检索，并获取带分数的结果
scored_results = rag_search_with_score(query, k=k)

# 基于余弦相似度的检索效果打分
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def cosine_similarity_score(query_embedding, doc_embeddings):
    """
    计算查询向量与文档向量之间的余弦相似度。
    参数:
        query_embedding (np.ndarray): 查询向量。
        doc_embeddings (np.ndarray): 文档向量矩阵，每行是一个文档的向量。
    返回:
        list: 每个文档的余弦相似度分数。
    """
    # 计算余弦相似度
    scores = cosine_similarity(query_embedding.reshape(1, -1), doc_embeddings)
    return scores.flatten().tolist()

# 获取查询向量
query_embedding = embedding_model.embed_query(query)

# 获取文档向量
doc_embeddings = np.array([embedding_model.embed_query(doc['content']) for doc in scored_results])

# 计算余弦相似度分数
cosine_scores = cosine_similarity_score(np.array(query_embedding), doc_embeddings)

# 将余弦相似度分数添加到文档元数据中
for doc, score in zip(scored_results, cosine_scores):
    doc['metadata']['cosine_score'] = score

# 输出每个文档的 L2 距离分数和余弦相似度分数
for idx, doc in enumerate(scored_results, 1):
    print(f"文档 {idx}: L2 距离分数 = {doc['metadata']['l2_score']}, 余弦相似度分数 = {doc['metadata']['cosine_score']}")

# 保存带 L2 距离分数和余弦相似度分数的检索结果到文件
output_dir = '../outputs/简单检索'
os.makedirs(output_dir, exist_ok=True)  # 创建 outputs 目录（如果不存在）
with open(f'../outputs/简单检索/rag_results_with_scores_k={k}.txt', 'w', encoding='utf-8') as f:
    for idx, doc in enumerate(scored_results, 1):
        f.write(f"\n===========结果{idx}==========:\n")
        f.write(f"内容:\n{doc['content']}\n")
        f.write(f"元数据: {doc['metadata']}\n")
        f.write(f"L2 距离分数: {doc['metadata']['l2_score']}\n")
        f.write(f"余弦相似度分数: {doc['metadata']['cosine_score']}\n")
print(f'带 L2 距离分数和余弦相似度分数的检索结果已保存到 ../outputs/简单检索/rag_results_with_scores_k={k}.txt')

文档 1: L2 距离分数 = 0.8710448741912842, 余弦相似度分数 = 0.5644775734272021
文档 2: L2 距离分数 = 0.8904027938842773, 余弦相似度分数 = 0.5547986173049412
文档 3: L2 距离分数 = 0.8913600444793701, 余弦相似度分数 = 0.5543199575628979
文档 4: L2 距离分数 = 0.8963077068328857, 余弦相似度分数 = 0.551846223802609
文档 5: L2 距离分数 = 0.8977255821228027, 余弦相似度分数 = 0.5511373129909685
文档 6: L2 距离分数 = 0.9011317491531372, 余弦相似度分数 = 0.5494341331280113
文档 7: L2 距离分数 = 0.9016575813293457, 余弦相似度分数 = 0.5491712251149357
文档 8: L2 距离分数 = 0.9032906889915466, 余弦相似度分数 = 0.5483546957951592
文档 9: L2 距离分数 = 0.9060183763504028, 余弦相似度分数 = 0.5469908216632857
文档 10: L2 距离分数 = 0.9064159393310547, 余弦相似度分数 = 0.5467920923993721
文档 11: L2 距离分数 = 0.9082849025726318, 余弦相似度分数 = 0.5458575784583303
文档 12: L2 距离分数 = 0.9112165570259094, 余弦相似度分数 = 0.5443917320261007
文档 13: L2 距离分数 = 0.9124183058738708, 余弦相似度分数 = 0.5437908743352812
文档 14: L2 距离分数 = 0.9145948886871338, 余弦相似度分数 = 0.5427025603661471
文档 15: L2 距离分数 = 0.9168858528137207, 余弦相似度分数 = 0.5415571073690935
文档 16: L2 距离分数 = 0.9

<font color=pink size=100>上下文压缩</font>

In [ ]:
#添加上下文压缩使用LLMChainExtractor
#现在让我们用 ContextualCompressionRetriever 包装我们的基本检索器。我们将添加一个 LLMChainExtractor，它将迭代最初返回的文档，并从每个文档中提取与查询相关的内容。
#问题：运行速度过于慢
#优点：效果更佳，减少了冗余信息


from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
k = 20  # 设置返回结果数量
# 创建上下文压缩检索器
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_db.as_retriever(search_kwargs={"k": k})
)

query = "请问保险产品有哪些？"
compressed_docs = compressor_retriever.invoke(query)


# 保存压缩后的结果到 outputs/上下文压缩/compressed_docs_k={k}.txt
import os
os.makedirs('../outputs/上下文压缩', exist_ok=True)  # 创建中文目录
with open(f'../outputs/上下文压缩/compressed_docs_k={k}.txt', 'w', encoding='utf-8') as f:
    for idx, doc in enumerate(compressed_docs, 1):
        f.write(f"\n===========结果{idx}==========:\n")
        f.write(f"压缩后的内容:\n{doc.page_content}\n")
        f.write(f"元数据: {doc.metadata}\n")
print(f'压缩后的结果已保存到 ../outputs/上下文压缩/compressed_docs_k={k}.txt')

压缩后的结果已保存到 ../outputs/上下文压缩/compressed_docs_k=20.txt


<font color=pink size=30>生成多个查询</font>

<font color=blue>没有提示词版本：多查询</font>

In [2]:
from langchain_ollama import OllamaLLM
from langchain.retrievers.multi_query import MultiQueryRetriever

# 采用 Ollama 的 LLM 模型千问
llm = OllamaLLM(model="qwen:7b")

# 使用 MultiQueryRetriever 从 LLM 创建多查询检索器
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_db.as_retriever(),  # 基于向量数据库的检索器
    llm=llm  # 使用的 LLM 模型
)

# 设置日志记录以查看生成的查询
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# 定义检索问题
question = "什么是宏利环球货币保障计划？"

# 调用多查询检索器获取相关文档
unique_docs = retriever_from_llm.get_relevant_documents(question)

# 获取生成的拓展问题
expanded_queries = retriever_from_llm.llm_chain.invoke(question)

# 输出检索到的文档数量
print(f"检索到的文档数量: {len(unique_docs)}")

# 保存检索结果和拓展问题到文件
output_dir = '../outputs/多查询检索'
import os
os.makedirs(output_dir, exist_ok=True)  # 如果文件夹不存在则创建
output_file = os.path.join(output_dir, 'multi_query_results_with_expanded_questions.txt')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("生成的拓展问题:\n")
    for idx, query in enumerate(expanded_queries, 1):
        f.write(f"{idx}. {query}\n")
    f.write("\n检索结果:\n")
    for idx, doc in enumerate(unique_docs, 1):
        f.write(f"\n===========结果{idx}==========:\n")
        f.write(f"内容: {doc.page_content}\n")
        f.write(f"元数据: {doc.metadata}\n")

# 打印保存结果的路径
print(f'检索结果和拓展问题已保存到 {output_file}')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21860\2957642352.py:22: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  unique_docs = retriever_from_llm.get_relevant_documents(question)
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 宏利环球货币保障计划具体是指什么服务？', '2. 这个名为宏利环球货币保障计划的项目，你能详细解释一下吗？', '3. 对于宏利环球货币保障计划这个金融产品，你可以提供一个简化的定义吗？']


检索到的文档数量: 6
检索结果和拓展问题已保存到 ../outputs/多查询检索\multi_query_results_with_expanded_questions.txt


<font color=blue>没有提示词版本：多查询+上下文压缩<font>

In [ ]:
#缺点：速度较慢，可能会导致响应时间延长

from langchain_ollama import OllamaLLM
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


# 采用 Ollama 的 LLM 模型千问
llm = OllamaLLM(model="qwen:7b")

# 使用 MultiQueryRetriever 从 LLM 创建多查询检索器
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_db.as_retriever(),  # 基于向量数据库的检索器
    llm=llm  # 使用的 LLM 模型
)


#在多查询检索器的基础上添加上下文压缩
compressor = LLMChainExtractor.from_llm(llm)
# 创建上下文压缩检索器
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever_from_llm
)


# 设置日志记录以查看生成的查询
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# 调用拟合检索器获取相关文档
query = "请问保险产品有哪些？"
compressed_docs = compressor_retriever.invoke(query)

# 获取生成的拓展问题
expanded_queries = retriever_from_llm.llm_chain.invoke(query)

# 输出检索到的文档数量
print(f"检索到的文档数量: {len(compressed_docs)}")

# 保存检索结果和拓展问题到文件
output_dir = '../outputs/多查询压缩检索'
import os
os.makedirs(output_dir, exist_ok=True)  # 如果文件夹不存在则创建
output_file = os.path.join(output_dir, 'compressed_docs_with_expanded_questions1.txt')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("生成的拓展问题:\n")
    for idx, query in enumerate(expanded_queries, 1):
        f.write(f"{idx}. {query}\n")
    f.write("\n检索结果:\n")
    for idx, doc in enumerate(compressed_docs, 1):
        f.write(f"\n===========结果{idx}==========:\n")
        f.write(f"内容: {doc.page_content}\n")
        f.write(f"元数据: {doc.metadata}\n")

# 打印保存结果的路径
print(f'检索结果和拓展问题已保存到 {output_file}')

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 您想了解的保险类型有哪些，以便做出选择？', '2. 除了基础的保险种类，您是否对特定领域的保险产品有所好奇？', '3. 您希望通过购买保险来解决哪些问题或风险？这样我可以帮您找到更匹配的产品。']


检索到的文档数量: 10
检索结果和拓展问题已保存到 ../outputs/多查询压缩检索\compressed_docs_with_expanded_questions.txt


<font color=blue>没有提示词版本：上下文压缩+多查询<font>

In [11]:

from langchain_ollama import OllamaLLM
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# 采用 Ollama 的 LLM 模型千问
llm = OllamaLLM(model="qwen:7b")
compressor = LLMChainExtractor.from_llm(llm)
#压缩器
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_db.as_retriever(search_kwargs={"k": k})
)
# 使用 MultiQueryRetriever 从 LLM 创建多查询检索器
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=compressor_retriever,  # 基于向量数据库的检索器
    llm=llm  # 使用的 LLM 模型
)

# 设置日志记录以查看生成的查询
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# 定义检索问题
question = "请问保险产品有哪些？"

# 调用多查询检索器获取相关文档
unique_docs = retriever_from_llm.get_relevant_documents(question)

# 获取生成的拓展问题
expanded_queries = retriever_from_llm.llm_chain.invoke(question)

# 输出检索到的文档数量
print(f"检索到的文档数量: {len(unique_docs)}")

# 保存检索结果和拓展问题到文件
output_dir = '../outputs/多查询压缩检索'
import os
os.makedirs(output_dir, exist_ok=True)  # 如果文件夹不存在则创建
output_file = os.path.join(output_dir, 'compressed_docs_with_expanded_questions2.txt')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("生成的拓展问题:\n")
    for idx, query in enumerate(expanded_queries, 1):
        f.write(f"{idx}. {query}\n")
    f.write("\n检索结果:\n")
    for idx, doc in enumerate(unique_docs, 1):
        f.write(f"\n===========结果{idx}==========:\n")
        f.write(f" 内容: {doc.page_content}\n")
        f.write(f"元数据: {doc.metadata}\n")

# 打印保存结果的路径
print(f'检索结果和拓展问题已保存到 {output_file}')

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 您想要了解哪些类型的保险产品？', '2. 如果您是投资者，想知道的是保险产品能带来什么收益吗？', '3. 您需要一份家庭保障计划，还是寻找投资增值的保险产品？']


检索到的文档数量: 59
检索结果和拓展问题已保存到 ../outputs/多查询压缩检索\compressed_docs_with_expanded_questions2.txt


<font color=blue>提示词版本：多查询</font>

In [ ]:
from typing import List
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
     template="""你是一名AI语言模型助手。你的任务是生成五个
给定用户问题的不同版本，用于从向量中检索相关文档
数据库。通过对用户问题产生多种观点，您的目标是帮助
用户克服了基于距离的相似性搜索的一些限制。
提供这些替代问题，并用换行符分隔。
    Original question: {question}""",
)

# Chain
llm_chain = QUERY_PROMPT | llm | output_parser

# 定义检索问题
question = "请问保险产品有哪些？"

# 创建多查询检索器
retriever = MultiQueryRetriever(
    retriever=vector_db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# 调用检索器获取相关文档和生成的拓展问题
unique_docs = retriever.get_relevant_documents(question)
expanded_queries = retriever.llm_chain.invoke(question)

# 输出检索到的唯一文档数量

print(f"检索到的文档数量: {len(unique_docs)}")

# 保存检索结果和拓展问题到文件
output_dir = '../outputs/多查询检索_提示词版本'
import os
os.makedirs(output_dir, exist_ok=True)  # 如果文件夹不存在则创建
output_file = os.path.join(output_dir, 'multi_query_results_with_expanded_questions_prompt_version.txt')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("生成的拓展问题:\n")
    for idx, query in enumerate(expanded_queries, 1):
        f.write(f"{idx}. {query}\n")
    f.write("\n检索结果:\n")
    for idx, doc in enumerate(unique_docs, 1):
        f.write(f"\n===========结果{idx}==========:\n")
        f.write(f"内容: {doc.page_content}\n")
        f.write(f"元数据: {doc.metadata}\n")

# 打印保存结果的路径
print(f'检索结果和拓展问题已保存到 {output_file}')

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 您能简要介绍一下各类保险产品吗？', '2. 我对保险不是很了解，能否给我列举一些常见的保险产品？', '3. 如果我想要购买一份保险，应该从哪些方面去考虑和选择产品？', '4. 您知道哪些保险公司提供的保险品种类丰富吗？可以分享一下吗？', '5. 购买保险时，有哪些因素会左右我选择的产品类型？']


INFO:langchain.retrievers.multi_query:Generated queries: ['1. 您能简要介绍一下各类保险产品吗？', '2. 我对保险不是很了解，能否给我列举一些常见的保险产品？', '3. 如果我想要购买一份保险，应该从哪些方面去考虑和选择产品？', '4. 您知道哪些保险公司提供的保险品种类丰富吗？可以分享一下吗？', '5. 购买保险时，有哪些因素会左右我选择的产品类型？']


检索到的文档数量: 17
检索结果和拓展问题已保存到 ../outputs/多查询检索_提示词版本\multi_query_results_with_expanded_questions_prompt_version.txt


<font color=blue>提示词版本：多查询+上下文压缩<font>

In [5]:
from typing import List
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
     template="""你是一名AI语言模型助手。你的任务是生成五个
给定用户问题的不同版本，用于从向量中检索相关文档
数据库。通过对用户问题产生多种观点，您的目标是帮助
用户克服了基于距离的相似性搜索的一些限制。
提供这些替代问题，并用换行符分隔。
    Original question: {question}""",
)

# Chain
llm_chain = QUERY_PROMPT | llm | output_parser

# 定义检索问题
question = "什么是宏利环球货币保障计划？"

# 创建多查询检索器
retriever_from_llm = MultiQueryRetriever(
    retriever=vector_db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# 在多查询检索器的基础上添加上下文压缩
compressor = LLMChainExtractor.from_llm(llm)
k = 20  # 设置返回结果数量
# 创建上下文压缩检索器
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever_from_llm
)

question = "保单的身故赔偿如何计算?"

# 调用检索器获取相关文档和生成的拓展问题
unique_docs =  compressor_retriever.get_relevant_documents(question)
expanded_queries = retriever_from_llm.llm_chain.invoke(question)

# 输出检索到的唯一文档数量

print(f"检索到的文档数量: {len(unique_docs)}")

# 保存检索结果和拓展问题到文件
output_dir = '../outputs/多查询检索_提示词版本_压缩检索'
import os
os.makedirs(output_dir, exist_ok=True)  # 如果文件夹不存在则创建
output_file = os.path.join(output_dir, 'compressed_docs_with_expanded_questions_prompt_version1.txt')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("生成的拓展问题:\n")
    for idx, query in enumerate(expanded_queries, 1):
        f.write(f"{idx}. {query}\n")
    f.write("\n检索结果:\n")
    for idx, doc in enumerate(unique_docs, 1):
        f.write(f"\n===========结果{idx}==========:\n")
        f.write(f"内容: {doc.page_content}\n")
        f.write(f"元数据: {doc.metadata}\n")

# 打印保存结果的路径
print(f'检索结果和拓展问题已保存到 {output_file}')

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 身故理赔金额是如何依据保单条款决定的？', '2. 计算一份保单的身故赔偿有什么具体步骤？', '3. 如果发生了身故，保单会自动计算并支付多少赔偿金？', '4. 一份保单的身故保险金额如何与投保人的年龄、健康状况等相关？', '5. 被保险人身故后，保险公司如何根据保险条款来确定和支付赔偿金额？']


检索到的文档数量: 10
检索结果和拓展问题已保存到 ../outputs/多查询检索_提示词版本_压缩检索\compressed_docs_with_expanded_questions_prompt_version1.txt


<font color=blue>提示词版本：上下文压缩+多查询<font>

In [3]:
from typing import List
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
k=1
compressor = LLMChainExtractor.from_llm(llm)
# 创建上下文压缩检索器
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_db.as_retriever(search_kwargs={"k": k})
)

class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
     template="""你是一名AI语言模型助手。你的任务是生成五个
给定用户问题的不同版本，用于从向量中检索相关文档
数据库。通过对用户问题产生多种观点，您的目标是帮助
用户克服了基于距离的相似性搜索的一些限制。
提供这些替代问题，并用换行符分隔。
    Original question: {question}""",
)

# Chain
llm_chain = QUERY_PROMPT | llm | output_parser

# 定义检索问题
question = "什么是宏利环球货币保障计划？"

# 创建多查询检索器
retriever = MultiQueryRetriever(
    retriever=compressor_retriever, llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# 调用检索器获取相关文档和生成的拓展问题
unique_docs = retriever.get_relevant_documents(question)
expanded_queries = retriever.llm_chain.invoke(question)

# 输出检索到的唯一文档数量

print(f"检索到的文档数量: {len(unique_docs)}")

# 保存检索结果和拓展问题到文件
output_dir = '../outputs/多查询检索_提示词版本_压缩检索'
import os
os.makedirs(output_dir, exist_ok=True)  # 如果文件夹不存在则创建
output_file = os.path.join(output_dir, 'compressed_docs_with_expanded_questions_prompt_version2.txt')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("生成的拓展问题:\n")
    for idx, query in enumerate(expanded_queries, 1):
        f.write(f"{idx}. {query}\n")
    f.write("\n检索结果:\n")
    for idx, doc in enumerate(unique_docs, 1):
        f.write(f"\n===========结果{idx}==========:\n")
        f.write(f"内容: {doc.page_content}\n")
        f.write(f"元数据: {doc.metadata}\n")

# 打印保存结果的路径
print(f'检索结果和拓展问题已保存到 {output_file}')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8924\359060397.py:48: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  unique_docs = retriever.get_relevant_documents(question)


KeyboardInterrupt: 